# Reference

This code is copied from the [hugging face translation tutorial](https://huggingface.co/docs/transformers/en/tasks/translation) 

In [1]:
import torch
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device


device(type='cuda')

In [2]:
books = load_dataset("opus_books", "en-fr")

In [3]:
books = books["train"].train_test_split(test_size=0.2)

In [5]:
books["train"][0]

{'id': '46495',
 'translation': {'en': 'Felton arose.', 'fr': 'Felton se leva.'}}

In [6]:
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [7]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "

In [8]:
def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [9]:
tokenized_books = books.map(preprocess_function, batched=True)

Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [10]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [11]:
metric = evaluate.load("sacrebleu")

In [12]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result



In [13]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_awesome_opus_books_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True, #change to bf16=True for XPU
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)



c:\Users\ethan\anaconda3\envs\nlp\lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [15]:
trainer.train()

  0%|          | 0/12710 [00:00<?, ?it/s]

{'loss': 2.0783, 'grad_norm': 1.2511531114578247, 'learning_rate': 1.9213217938631003e-05, 'epoch': 0.08}
{'loss': 2.0098, 'grad_norm': 1.7367422580718994, 'learning_rate': 1.8426435877261997e-05, 'epoch': 0.16}
{'loss': 1.9838, 'grad_norm': 1.2962981462478638, 'learning_rate': 1.7639653815893e-05, 'epoch': 0.24}
{'loss': 1.936, 'grad_norm': 1.27520751953125, 'learning_rate': 1.6852871754524e-05, 'epoch': 0.31}
{'loss': 1.9218, 'grad_norm': 1.3131723403930664, 'learning_rate': 1.6067663257277734e-05, 'epoch': 0.39}
{'loss': 1.8979, 'grad_norm': 1.2872899770736694, 'learning_rate': 1.5280881195908735e-05, 'epoch': 0.47}
{'loss': 1.8931, 'grad_norm': 1.375, 'learning_rate': 1.4494099134539735e-05, 'epoch': 0.55}
{'loss': 1.8675, 'grad_norm': 1.5202994346618652, 'learning_rate': 1.3707317073170734e-05, 'epoch': 0.63}
{'loss': 1.8796, 'grad_norm': 1.4575570821762085, 'learning_rate': 1.292210857592447e-05, 'epoch': 0.71}
{'loss': 1.8787, 'grad_norm': 1.3211054801940918, 'learning_rate': 1.

c:\Users\ethan\anaconda3\envs\nlp\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/1589 [00:00<?, ?it/s]

{'eval_loss': 1.63423752784729, 'eval_bleu': 5.458, 'eval_gen_len': 17.6357, 'eval_runtime': 410.8565, 'eval_samples_per_second': 61.863, 'eval_steps_per_second': 3.868, 'epoch': 1.0}
{'loss': 1.8521, 'grad_norm': 1.2078088521957397, 'learning_rate': 9.778127458693943e-06, 'epoch': 1.02}
{'loss': 1.8369, 'grad_norm': 1.173573613166809, 'learning_rate': 8.991345397324942e-06, 'epoch': 1.1}
{'loss': 1.8406, 'grad_norm': 1.3431209325790405, 'learning_rate': 8.20456333595594e-06, 'epoch': 1.18}
{'loss': 1.8158, 'grad_norm': 1.3518027067184448, 'learning_rate': 7.4177812745869405e-06, 'epoch': 1.26}
{'loss': 1.8377, 'grad_norm': 1.4119102954864502, 'learning_rate': 6.630999213217939e-06, 'epoch': 1.34}
{'loss': 1.8333, 'grad_norm': 1.1311908960342407, 'learning_rate': 5.844217151848939e-06, 'epoch': 1.42}
{'loss': 1.8314, 'grad_norm': 1.205683708190918, 'learning_rate': 5.057435090479937e-06, 'epoch': 1.49}
{'loss': 1.8315, 'grad_norm': 1.2215049266815186, 'learning_rate': 4.270653029110937

c:\Users\ethan\anaconda3\envs\nlp\lib\site-packages\transformers\generation\utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/1589 [00:00<?, ?it/s]

{'eval_loss': 1.6113057136535645, 'eval_bleu': 5.635, 'eval_gen_len': 17.6162, 'eval_runtime': 439.6295, 'eval_samples_per_second': 57.815, 'eval_steps_per_second': 3.614, 'epoch': 2.0}
{'train_runtime': 2103.6582, 'train_samples_per_second': 96.658, 'train_steps_per_second': 6.042, 'train_loss': 1.8729620908771698, 'epoch': 2.0}


TrainOutput(global_step=12710, training_loss=1.8729620908771698, metrics={'train_runtime': 2103.6582, 'train_samples_per_second': 96.658, 'train_steps_per_second': 6.042, 'total_flos': 4984546558083072.0, 'train_loss': 1.8729620908771698, 'epoch': 2.0})

In [ ]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."